In [1]:
from SQLAgent import *
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash', temprature = 0)
db_name = "customer_database"
table_name = "CRM"
question = "Whats our most expensive product and what is its price? Which country has it been bought the most?"

df = sql_agent_response(
    question = question, 
    llm = llm, 
    db_name = db_name, 
    table_name = table_name)
df

SELECT T1.Description, T1.Price, T1.Country, COUNT(T1.Description) AS PurchaseCount FROM (SELECT Description, Price, Country FROM CRM ORDER BY Price DESC LIMIT 1) AS T1 INNER JOIN CRM AS T2 ON T1.Description = T2.Description GROUP BY T1.Description, T1.Price, T1.Country ORDER BY PurchaseCount DESC LIMIT 1
[('Manual', 25111.09, 'United Kingdom', 854)]


,Description,Price,Country,PurchaseCount
0,Manual,25111.09,United Kingdom,854


In [31]:
conn = sqlite3.connect('customer_database.db')
cursor = conn.cursor()
cursor.execute("PRAGMA table_info([CRM]);")
# result = pd.DataFrame(cursor.fetchall())
cursor.fetchall()

[(0, 'Invoice', 'TEXT', 0, None, 0),
 (1, 'StockCode', 'TEXT', 0, None, 0),
 (2, 'Description', 'TEXT', 0, None, 0),
 (3, 'Quantity', 'INTEGER', 0, None, 0),
 (4, 'InvoiceDate', 'TIMESTAMP', 0, None, 0),
 (5, 'Price', 'REAL', 0, None, 0),
 (6, 'Customer_ID', 'REAL', 0, None, 0),
 (7, 'Country', 'TEXT', 0, None, 0)]

In [1]:
import sqlite3
import pandas as pd

In [6]:
conn = sqlite3.connect("customer_database.db")
cursor = conn.cursor()

In [5]:
df = pd.read_excel('../online_retail_II.xlsx')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [17]:
df.columns = [i.replace(' ','_') for i in df.columns]
df.to_sql('CRM',conn,if_exists='replace',index = False)

525461

In [18]:
conn.commit()
conn.close()

In [2]:
import sqlite3
import pandas as pd
def query_db(query):
    conn = sqlite3.connect('customer_database.db')
    cursor = conn.cursor()
    cursor.execute(query)
    result = pd.DataFrame(cursor.fetchall())
    column_names = [description[0] for description in cursor.description]
    result.columns = column_names
    if 'InvoiceDate' in result.columns:
        result['InvoiceDate'] = pd.to_datetime(result['InvoiceDate'])
    conn.close()
    return result


In [3]:
df= query_db("select * from CRM where InvoiceDate > Datetime('2009-12-01 07:45:00') and InvoiceDate < Datetime('2009-12-01 07:48:00') ")

True

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Markdown
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temprature = 0)
# display(Markdown(llm.invoke("What is life?").content))

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
question = "What are the two most bought products and what percentage of the total population do they represent? Give me the count of each of these and the percentage."



sql_prompt = PromptTemplate(
    template = """
    System: You are an SQL Agent.
    You have access to a single database called {db_name}
    This database has only a single table called {table_name}.
    You will be prompted with a question from a user and your job is to write an SQL query that will retrieve the relevant data from the database.
    Assign proper aliases in your query.
    Additional info, you are using SQLite, so use the appropriate sql functions when required
    The columns of the CRM table are as follows:
    {columns}
    In your response, only return an SQL query in JSON format with nothing before or after it.
    ===========================================================================
    Question: {question}
    """,
    input_variables = ['db_name','table_name','columns','question']
)
sql_agent = sql_prompt | llm | JsonOutputParser()

query = sql_agent.invoke({
    'db_name':'customer_database',
    'table_name':'CRM',
    'columns':df.columns,
    'question':question
})['query']
print(query)
query_db(query)

SELECT T2.Description, SUM(T1.Quantity) AS TotalQuantity, CAST(SUM(T1.Quantity) AS REAL) * 100 / (SELECT SUM(Quantity) FROM CRM) AS Percentage FROM CRM AS T1 INNER JOIN CRM AS T2 ON T1.StockCode = T2.StockCode GROUP BY T2.Description ORDER BY TotalQuantity DESC LIMIT 2
